# Hacer predicciones en base a la ruta de una imagen (Google Colab)
A01369630 Julieta Itzel Pichardo Meza

# Guía rápida para usar este Notebook en Google Colab

NOTA: Este Notebook ya se encuentra en Google Colab, en el siguiente link:

https://drive.google.com/file/d/1oB_5Ic-DtZsqhLhFF0_mg0etrOPXWwQl/view?usp=sharing

## 1. Subir archivos a Google Drive

1. Guardar en Google Drive:
   - El archivo de pesos del modelo v3 (`.pth`), por ejemplo:  
     `clasificador_perros_gatos_v3.pth`
   - Algunas imágenes de prueba para hacer predicciones.

2. Organizar en carpetas, por ejemplo:
   - `Mi unidad/perros_gatos_modelos/` → archivo `.pth`
   - `Mi unidad/perros_gatos_tests/` → imágenes `.jpg` o `.png`

## 2. Ejecutar las celdas del Notebook

1. **Celda 1 – Montar Drive**  
   - Ejecutar la celda que contiene:
     ```python
     from google.colab import drive
     drive.mount('/content/drive')
     ```
   - Autorizar el acceso a Google Drive cuando Colab lo pida.

2. **Celda 2 – Imports, dispositivo y transforms**  
   - Verificar que imprima algo como:
     ```
     Usando dispositivo: cuda
     ```
     o
     ```
     Usando dispositivo: cpu
     ```

3. **Celda 3 – Modelo v3 y carga de pesos**  
   - Ajustar la variable `weights_path` para que apunte a la ruta correcta del archivo `.pth` en Drive.
   - Verificar que imprima:
     ```
     Modelo v3 cargado correctamente desde: ...
     ```

4. **Celda 4 – Funciones de predicción**  
   - Define las funciones `predict_image_no_plot` y `predict_from_path_string`.
   - No debería imprimir nada, salvo que haya un error.

5. **Celda 5 – Probar una imagen**  
   - Ajustar la variable:
     ```python
     ruta_imagen = "/content/drive/MyDrive/perros_gatos_tests/nueve.jpg"
     ```
     según la ruta real de tu imagen en Drive.
   - Ejecutar la celda.  
   - Debería mostrar en la salida algo similar a:
     ```
     >>> Predicción: dog (probabilidad: 0.932)
     ```

## 3. Probar otras imágenes

Para hacer nuevas pruebas:

- Solo necesitas cambiar el valor de `ruta_imagen` en la última celda:
  ```python
  ruta_imagen = "/content/drive/MyDrive/perros_gatos_tests/otra_imagen.jpg"


In [ ]:
# Celda 1: Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')

print("Drive montado en /content/drive")


In [ ]:
# Celda 2: Imports, configuración de dispositivo y transforms

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from pathlib import Path

# Usar GPU si está disponible en Colab, si no, CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# Clases del modelo (ajusta si las tienes en otro orden)
class_names = ["cat", "dog"]
num_classes = len(class_names)

# Transformaciones de inferencia (deben coincidir con las de entrenamiento de la v3)
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # o el tamaño que usaste al entrenar
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # mean de ImageNet
        std=[0.229, 0.224, 0.225]    # std de ImageNet
    ),
])


In [ ]:
# Celda 3: Definir modelo v3 (ResNet) y cargar pesos desde Google Drive

def create_model_v3(num_classes=2):
    """
    Crea el modelo de la versión 3 usando ResNet preentrenada
    y reemplaza la capa fully-connected final.
    """
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

# Ruta al archivo .pth en tu Google Drive (AJÚSTALA)
weights_path = Path("/content/drive/MyDrive/perros_gatos_modelos/clasificador_perros_gatos_v3.pth")

if not weights_path.is_file():
    raise FileNotFoundError(f"No se encontró el archivo de pesos en: {weights_path}")

model_v3 = create_model_v3(num_classes=num_classes)

# Cargar los pesos (Colab soporta bien GPU, así que mapeamos al dispositivo elegido)
state_dict = torch.load(weights_path, map_location=device)
model_v3.load_state_dict(state_dict)

model_v3.to(device)
model_v3.eval()

print(f"Modelo v3 cargado correctamente desde: {weights_path}")


In [ ]:
# Celda 4: Función de predicción sin gráfica + wrapper

def predict_image_no_plot(image_path, model, transform, class_names, device):
    """
    Carga una imagen desde image_path, aplica las transformaciones de inferencia,
    ejecuta el modelo y devuelve (pred_label, pred_prob).
    """
    image_path = Path(image_path)
    if not image_path.is_file():
        raise FileNotFoundError(f"No se encontró la imagen: {image_path}")

    # Cargar imagen
    img = Image.open(image_path).convert("RGB")

    # Aplicar transformaciones
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)  # (1, C, H, W)
    img_tensor = img_tensor.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        prob, pred_idx = torch.max(probs, dim=1)

    pred_label = class_names[pred_idx.item()]
    pred_prob = prob.item()

    return pred_label, pred_prob


def predict_from_path_string(path_str):
    """
    Wrapper sencillo: recibe una ruta de imagen como string,
    llama a predict_image_no_plot y muestra el resultado por pantalla.
    """
    try:
        pred_label, pred_prob = predict_image_no_plot(
            image_path=path_str,
            model=model_v3,
            transform=inference_transform,
            class_names=class_names,
            device=device
        )
        print(f">>> Predicción: {pred_label} (probabilidad: {pred_prob:.3f})")
    except Exception as e:
        print(f"Error al procesar la imagen: {e}")


In [ ]:
# Celda 5: Probar la predicción con una imagen en Google Drive

ruta_imagen = "/content/drive/MyDrive/perros_gatos_tests/nueve.jpg"  # AJUSTA ESTA RUTA

predict_from_path_string(ruta_imagen)
